# Final Project Check-in 2018-11-16

## Group Name: Lambda

### Student Names
1. Jian Wang
2. Chong Geng
3. Alan Perry
4. Divya Bhargavi
5. Robert Sandor

## Load Data

In [1]:
from collections import defaultdict
import math
from math import sqrt
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import operator
import re
from scipy import spatial
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import string
import time

In [2]:
def make_dictionary(file):
    '''
    Initiate the glove model as a dictionary
    input: A String which is a file in the project directory
    returns: A dictionary with item = word : 300 d list
    
    :param file:            the filepath string of the dictionary
    :returns:               a dictionary with words as keys 
                            and 300d vectors as values
    '''
    vecs = defaultdict(lambda: np.zeros(shape=(300,1)))
    with open(file) as f:
        lines = f.readlines()
        for word_and_vec in lines:
            elems = word_and_vec.strip().split(' ')
            word = elems[0]
            vec = np.array(elems[1:], dtype=float)
            vecs[word] = vec
    return vecs

In [3]:
def split_dictionary():
    """
    firstly, I split the dictionary into a wordlist and a matrix.
    returns a list of words and 
    a 2d matrix of the normalized word vectors
    
    :returns:               the words and matrix associated with
                            the glove dictionary
    """
    wordlist = []
    matrix = []
    with open(glove_file) as f:
        lines = f.readlines()
        for word_and_vec in lines:
            wordvec = np.array([float(x) for x in word_and_vec.split()[1:]])    
            matrix.append(wordvec / np.linalg.norm(wordvec))
            wordlist.append(word_and_vec.split()[0])
        matrix = np.array(matrix)
    return wordlist, matrix

def unique_words(train_df):
    """
    I then obtain the unique words that appear in the search_term.
    
    :param train_df:        the training set Pandas dataframe
    :returns:               a list of unique words from search terms
                            that have been stripped of numbers, symbols, etc.
    """
    cleaned = list(train_df['cleaned_terms'])
    all_words = []
    for t in cleaned:
        all_words += t.split(' ')

    return list(set(all_words))[1:]

def find_nearest_neighbors(filename, cleaned_set, matrix, wordlist, dictionary):
    """
    here I count the cos_distance of each word that is in the cleaned_set.
    the output file looks like (each line): w0, w1, w2, w3, w4,
    i didn't print the distance, just the neighbour words
    this will take couple of minutes.
    
    :param filename:        a string representing the filename to write to
    :param cleaned_set:     a list of search terms that have 
                            been stripped of numbers, symbols, etc.
    :param matrix:          a 2d Numpy array of the word vectors in wordlist
    :param wordlist:        a list of words from the glove dictionary
    :param dictionary:      a dictionary with words as keys 
                            and 300d vectors as values
    """ 
    output_string = ''
    
    for word in cleaned_set:
        dots = matrix.dot(dictionary[word])
        close_index_vec = np.argsort(dots)
        for i in range(5):
            output_string += wordlist[int(close_index_vec[-1-i])] + ','
        output_string += '\n'
        
    f = open(filename, "w")
    f.write(output_string)
    f.close()

def get_all_terms_neighbors(dictionary, cleaned):
    """
    terms_neighbour is the list which stores the top 4 neighbours of each searching_terms. 
    for example, if the searching term is: cleaned[0]='w1_w2', 
    then the terms_neighbour[0]='n11_n12_n13_n14_n21_n22_n23_n24'.
    
    :param dictionary:      a dictionary
    :param cleaned:         a list of search terms that have
                            been stripped of numbers, symbols, etc.
    :returns:               a list of concatenated words that are neighbors
                            of the 'cleaned' terms
    """
    terms_neighbour = []
    for i in range(len(cleaned)):
        neighbours = ''
        if cleaned[i] != '':
            words = cleaned[i].split(' ')
            for w in words:
                neighbours = neighbours + dictionary[w] + ' '
        terms_neighbour.append(neighbours)
    return terms_neighbour

def build_dictionary(file):
    """
    based on the above output file, I then built a dictionary;
    this dictionary stores each word (as key) with its top 4 neighbour words (as value) 
    
    :param file:            the file containing the list of strings of neighbors
    :returns:               a dictionary with words as keys 
                            and 4 neighbors of that word as values
    """
    k_dic = defaultdict(lambda: '')
    with open(file) as f:
        lines = f.readlines()
        for line in lines:
            words = line.strip().split(',')
            k_dic[words[0]] = words[1] + ' ' + words[2] + ' ' + words[3] + ' ' + words[4]
    return k_dic

def clean_term_in_doc(terms, title):
    """
    This cleans the given terms in the specified document
    
    :param terms:           a list of unique search terms
    :param title:           a list of titles of products
    :return:                a list of the counts of the 
                            cleaned terms within a product's title
    """
    count = np.zeros(len(terms))
    for i in range(len(terms)):
        if not pd.isnull(terms[i]): 
            title[i] = title[i].lower()
            for term in terms[i].split(' '):
                if term in title[i].split(' '):
                    count[i] += 1
    return count

def get_length(column):
    """
    This calculates and returns the number of words
    for each row in a specified column
    
    :param column:          the feature/attribute which
                            will have its words counted
    :returns:               a column with the count of 
                            words in each string
    """
    length = np.zeros(len(column))
    for index in range(len(column)):
        if not pd.isnull(column[index]):
            length[index] = len(column[index].split(' '))
    return length

def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    
    :param text:            a string
    :returns:               the same string stripped of numbers,
                            tabs, newline characters, and punctuation
    """
    stops = list(stop_words.ENGLISH_STOP_WORDS)
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words = nopunct.split(" ")
    words = [w for w in words if (len(w) > 2 and (w not in stops))]  # ignore a, an, to, at, be, ...
    return words

In [4]:
def letter_prob(phrases):
    """
    :param phrases:         a list of strings of text
    :returns:               a list of dictionaries of probabilities for characters in the text 
    """
    letter_counters = []
    for phrase in phrases:
        letter_count = defaultdict(lambda: 0)
        for char in phrase:
            if char.isalpha():
                if char in letter_count:
                    letter_count[char] += 1
                else:
                    letter_count[char] = 1
        letter_counters.append(letter_count)
        
        total_count = float(sum(list(letter_count.values())))
    
        for key in letter_count.keys():
            letter_count[key] = letter_count[key] / total_count
            
    return letter_counters

def calculate_kl_divergences(P, Q):
    """
    :param P:               a list of dictionaries of probabilities for a distribution
    :param Q:               a list of dictionaries of probabilities for a second distribution
    :returns:               a list of the kl-divergences between P and Q
    """
    kl_divergences = []
    for idx, q in enumerate(Q):
        kl_divergence = 0
        for key in q.keys():
            if P[idx][key] != 0:
                kl_divergence += P[idx][key] * math.log2(P[idx][key] / q[key])
        kl_divergence *= -1
        kl_divergences.append(kl_divergence)
    return kl_divergences

def calculate_entropy(probs_list):
    """
    :param probs_list:      a list of dictionaries in which the values are probabilities
    :returns:               a list of entropies calculated for the given probs_list
    """
    entropies = []
    for distribution in probs_list:
        entropy = 0
        for key in distribution.keys():
            entropy += distribution[key] * math.log2(distribution[key])
        entropy *= -1
        entropies.append(entropy)
    return entropies

In [5]:
def longest_common_subsequence(X , Y): 
    """
    :param X:               a list of strings of text
    :param Y:               a list of strings of text
    :returns:               a list of the integer length of the longest common subsequence 
                            between the strings
    """
    lcs = []
    
    for idx, x in enumerate(X):
        m = len(x) 
        n = len(Y[idx]) 
 
        L = [[None]*(n+1) for i in range(m+1)] 

        for i in range(m+1): 
            for j in range(n+1): 
                if i == 0 or j == 0 : 
                    L[i][j] = 0
                elif x[i-1] == Y[idx][j-1]: 
                    L[i][j] = L[i-1][j-1]+1
                else: 
                    L[i][j] = max(L[i-1][j] , L[i][j-1]) 
        lcs.append(L[m][n])
                    
    return lcs

In [14]:
def calculate_jaccard_index(text_1, text_2):
    """
    :param text_1:         a list of strings of text
    :param text_2:         a second list of strings of text
    :returns:              a list of jaccard indices between 
                           the strings of text provided
    """
    jaccard_indices = []
    for text in zip(text_1, text_2):
        tokens_1 = set(tokenize(text[0]))
        tokens_2 = set(tokenize(text[1]))
        intersection_ = tokens_1.intersection(tokens_2)
        union_ = tokens_1.union(tokens_2)
        jaccard_indices.append(len(list(intersection_)) / float(len(list(union_))))
    return jaccard_indices

In [7]:
def feature_engineering(train_df, products_df, dictionary):
    """
    Adds the following features to the training set dataframe: 
    * clean_length: the count of words in the 'cleaned' search terms
    * title_length: the count of words in the 'cleaned' title
    * desc_length: the count of words in the 'cleaned' description
    * clean_terms_in_title: the number of time 
    any of the words in clean_terms appears in the title
    * clean_terms_in_desc: the number of time 
    any of the words in clean_terms appears in the description
    * neighbours_in_title: the count of the appearance of the 
    words closest to the search terms in the title
    * neighbours_in_desc: the count of the appearance of the 
    words closest to the search terms in the description
    
    :param train_df:        the training set Pandas dataframe
    :param products_df:     the product descriptions dataframe
    :param dictionary:      the glove dictionary
    :returns:               the modified dataframe with the additional features
    """
    # join the dataframes together
    train_df = train_df.set_index('product_uid').join(products_df.set_index('product_uid'))
    train_df = train_df.reset_index()
    
    # "clean" the search terms of numbers and stop words
    search_terms = train_df['search_term']
    cleaned_terms = [' '.join(tokenize(search_term)) for search_term in search_terms]
    train_df['cleaned_terms'] = cleaned_terms
    
    cleaned = list(train_df['cleaned_terms'])
    title = list(train_df['product_title'])
    desc = list(train_df['product_description'])
    


    # set up the calculations for finding the nearest neighbors
    wordlist, matrix = split_dictionary()
    cleaned_set = unique_words(train_df)
    find_nearest_neighbors('glove_neighbour_no_w.txt', cleaned_set, matrix, wordlist, dictionary)
    k_dict = build_dictionary('glove_neighbour_no_w.txt')
    terms_neighbour = get_all_terms_neighbors(k_dict, cleaned)
    train_df['terms_neighbour'] = terms_neighbour
    
    # create the features to be used in the model
    train_df['clean_length'] = get_length(cleaned)
    train_df['title_length'] = get_length(title)
    train_df['desc_length'] = get_length(desc)
    train_df['clean_terms_in_title'] = clean_term_in_doc(cleaned, title)
    train_df['clean_terms_in_desc'] = clean_term_in_doc(cleaned, desc)
    train_df['neighbours_in_title'] = clean_term_in_doc(terms_neighbour, title)
    train_df['neighbours_in_desc'] = clean_term_in_doc(terms_neighbour, desc)
    
    # added extra features, including lcs, entropy and jaccard index
    train_df['lcs_title'] = longest_common_subsequence(cleaned, title)
    train_df['lcs_desc'] = longest_common_subsequence(cleaned, desc)
    train_df['search_terms_entropy'] = calculate_entropy(letter_prob(cleaned))
    train_df['title_entropy'] = calculate_entropy(letter_prob(title))
    train_df['jaccard_index_title'] = calculate_jaccard_index(title, cleaned)
    train_df['jaccard_index_desc'] = calculate_jaccard_index(desc, cleaned)
    
    return train_df

In [8]:
products = pd.read_csv('product_descriptions.csv')
train = pd.read_csv('train.csv', encoding='ISO-8859-1')

In [9]:
glove_file = 'glove.6B.300d.txt'
glove_dic = make_dictionary(glove_file)

In [16]:
modified_train = feature_engineering(train, products, glove_dic)

## Fit scikit-learn model

In [17]:
X_train = modified_train[['clean_length', 'title_length', 
                          'desc_length', 'clean_terms_in_title', 
                          'clean_terms_in_desc', 'neighbours_in_title',
                         'neighbours_in_desc', 'title_entropy',
                         'search_terms_entropy', 'lcs_title', 'lcs_desc',
                         'jaccard_index_title', 'jaccard_index_desc']]
y_train = modified_train[['relevance']]

In [18]:
# since we can't see the relevancy scores of the test set,
# I decided to split the training set 
train_data, test_data, train_target, test_target = train_test_split(X_train,
                                                                        y_train)

In [19]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(train_data, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [20]:
predicted = lin_reg_model.predict(test_data)
print(predicted[:5])
print(test_target[:5])

[[2.63908569]
 [2.2400152 ]
 [2.47129219]
 [2.31350679]
 [2.50350051]]
       relevance
45055       3.00
64720       1.00
71594       3.00
61619       2.33
52166       2.67


In [ ]:
# the following code was to test out AdaBoost and Bagging Trees
# they eventually turned out to be less accurate than LinearRegression
# but I provide them here to show what was done and the thought process behind it

# from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
# from sklearn.model_selection import GridSearchCV

# models = [AdaBoostRegressor(),
#          BaggingRegressor()]

# grid_params = [{'n_estimators': range(1, 20, 3),
#               'loss': ['linear', 'exponential'],
#               'learning_rate': np.linspace(start=0.45, stop=.7, num=5)},
#               {'n_estimators': range(10, 20, 3),
#               'max_samples': np.linspace(start=0.2, stop=1.0, num=5),
#               'max_features': np.linspace(start=0.25, stop=1.0, num=4)}]
# best_models = []
# for idx, model in enumerate(models):
#     gs = GridSearchCV(estimator=model,
#                      param_grid=grid_params[idx],
#                      scoring='neg_mean_squared_error')
#     gs.fit(train_data, train_target)
#     best_models.append((gs.best_score_, gs.best_params_, models[idx]))
# # {'learning_rate': 0.5499999999999999, 'loss': 'linear', 'n_estimators': 3}
# # {'learning_rate': 0.6375, 'loss': 'exponential', 'n_estimators': 4}

In [ ]:
# best_models

In [ ]:
# adaboost = AdaBoostRegressor(**best_models[0][1])
# adaboost.fit(train_data, train_target)
# adaboost_predicted = adaboost.predict(test_data)

# bagging = BaggingRegressor(**best_models[1][1])
# bagging.fit(train_data, train_target)
# bagging_predicted = bagging.predict(test_data)

In [ ]:
# after testing out Adaboost and learning that it performed
# worse than LinearRegression, I decided to test out other
# linear models; one of them (Lasso I believe) performed
# equivalently and the others performed worse than LinearRegression

# from sklearn.linear_model import Lasso, Ridge, ElasticNet

# models_linear = [Lasso(),
#                  Ridge(),
#                  ElasticNet()]

# grid_params_linear = [{'alpha': np.linspace(start=0.2, stop=1.0, num=5)},
#                       {'alpha': np.linspace(start=0.2, stop=1.0, num=5),
#                       'solver': ['svd', 'lsqr', 'sag', 'auto']},
#                       {'alpha': np.linspace(start=0.2, stop=1.0, num=5),
#                       'l1_ratio': np.linspace(start=0.2, stop=1.0, num=5),
#                       'selection': ['cyclic', 'random']}]
# best_models_linear = []
# for idx, model in enumerate(models_linear):
#     gs = GridSearchCV(estimator=model,
#                      param_grid=grid_params_linear[idx],
#                      scoring='neg_mean_squared_error')
#     gs.fit(train_data, train_target)
#     best_models_linear.append((gs.best_score_, gs.best_params_, models_linear[idx]))

In [ ]:
# lasso = Lasso(**best_models_linear[0][1])
# lasso.fit(train_data, train_target)
# lasso_predicted = lasso.predict(test_data)

# ridge = Ridge(**best_models_linear[1][1])
# ridge.fit(train_data, train_target)
# ridge_predicted = ridge.predict(test_data)

# elastic_net = ElasticNet(**best_models_linear[2][1])
# elastic_net.fit(train_data, train_target)
# elastic_net_predicted = elastic_net.predict(test_data)

In [ ]:
# print(best_models_linear)

In [ ]:
# this was a rough test of RandomForest
# and indicated that it performed better than Linear Regression

# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100)
# rf.fit(train_data, train_target)
# rf_predicted = rf.predict(test_data)

## Evaluation Metric

In [21]:
rmse_lin_reg = sqrt(mean_squared_error(predicted, test_target))

# the benchmark was ~ rank 1681 on the Kaggle Leaderboard
# https://www.kaggle.com/c/home-depot-product-search-relevance/leaderboard
# original set of features from checkin template: .5102
# with only kl-divergence: .5130
# with only title_entropy and search_entropy: .5086
# with title_entropy, search_entropy, lcs_title, lcs_desc, jaccard_title, and jaccard_desc: .4971
print(f"{rmse_lin_reg:.4f}")

0.4971


In [ ]:
# rmse_adaboost = sqrt(mean_squared_error(adaboost_predicted, test_target))
# print(f"{rmse_adaboost:.4f}")

In [ ]:
# rmse_bagging = sqrt(mean_squared_error(bagging_predicted, test_target))
# print(f"{rmse_bagging:.4f}")

In [ ]:
# rmse_lasso = sqrt(mean_squared_error(lasso_predicted, test_target))
# print(f"{rmse_lasso:.4f}")

# rmse_ridge = sqrt(mean_squared_error(ridge_predicted, test_target))
# print(f"{rmse_ridge:.4f}")

# rmse_elastic_net = sqrt(mean_squared_error(elastic_net_predicted, test_target))
# print(f"{rmse_elastic_net:.4f}")

In [ ]:
# rmse_rf = sqrt(mean_squared_error(rf_predicted, test_target))
# print(f"{rmse_rf}:.4f")